# People detection with YOLO model

In [1]:
# prerequisites:

# !pip install torch
# !pip install imutils
# !pip install argparse
# !pip install imutils
# !pip install cv2
# !pip install numpy
# !pip install ultralytics
# !pip install -U ultralytics sahi

## Libraries import + initial settings

In [2]:
import torch
import argparse
import imutils
import time
import cv2
import numpy as np

from ultralytics import YOLO

from sahi import AutoDetectionModel
from sahi.predict import get_prediction

from sahi.predict import get_sliced_prediction

## Path to the video file

In [3]:
path_to_video = r"C:\Users\aksen\OneDrive\Desktop\people detection video\crowd.mp4"

## Downloading weigths for the detection model

In [4]:
model = YOLO("yolov8m.pt")

detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path="yolov8m.pt",
    confidence_threshold=0.3,
    device="cpu"
)

## Reading frames from the video and run the model on it

In [5]:
video = cv2.VideoCapture(path_to_video)

In [8]:
# video recording
recording = cv2.VideoWriter(filename=r"C:\Users\aksen\OneDrive\Desktop\people detection video\detected_video.mp4",
    fourcc=-1,
    fps=15,
    frameSize=(1920, 1080)
)


# iterating over the video frames

flag, frame = video.read()

while flag:

    
    # detecting people
    
    predictions = get_sliced_prediction(
        frame,
        detection_model,
        slice_height=512,
        slice_width=512,
        overlap_height_ratio=0.1,
        overlap_width_ratio=0.1,
        verbose=0,
    )
    
    coco_results = predictions.to_coco_predictions()
    

    # filtering out irrelevant objects and drawing areas with detected people on the frame    
    
    for c in coco_results:
        if c['category_name'] == 'person':
            
            xA, yA = c['bbox'][0], c['bbox'][1]
            w, h = c['bbox'][2], c['bbox'][3]
            
            cv2.rectangle(frame, (int(xA), int(yA)), (int(xA+w), int(yA+h)), (0, 255, 0), 2)
    
    
    # Drawing the frame itself
    
    cv2.imshow('Frame', frame)
    recording.write(frame)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
    
    
    # Continuing with the next frame
    
    flag, frame = video.read()

cv2.destroyAllWindows()

# Results and conclusions

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

## Representation of the model detection

In [ ]:
Image(url= "https://github.com/naketu/People_recognition/blob/master/crowd_detected.png?raw=true", width=1000, height=1000)

Комментарии в коде я оставлял на английском языке, чтобы не переключаться на русскую раскладку во время программирования. Надеюсь это не повлияет на результат)

Модель с использованием SAHI работает гораздо лучше чем без. Однако из за дробления картинки на маленький части, модель иногда выделяет части одного человека как несколько разных. При этом модель хорошо определила как близких так и дальних людей.

Можно сделать вывод что точность модели, как и ее полнота, довольно хороши. Чтобы улучшить результат нужно поработать над распознаванием людей в левом углу картинки (например добавить рамки к картинке), а также настроить размер окна работы алгоритма SAHI. Вероятно, стоит его увеличить чтобы убрать проблемы с ложным распознаванием нескольких частей человека как отдельных объектов.

Для улучшения скорости работы модели стоит использовать вычисления на видеокартах вместо процессора - однако у меня внезапно полетела cuda, и перенести вычисления на видеокарту не удалось.